# Моделирование рядов

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import random
%matplotlib inline
plt.style.use('seaborn-poster')
import warnings
warnings.filterwarnings("ignore")

Генерируем 5 наборов рядов:
* Белый шум
* Линейный тренд
* Линейный тренд + белый шум
* Линейный тренд с изломами
* Линейный тренд с изломами + белый шум

В каждом таком наборе 100 рядов по 1000 точек каждый.

Параметры белого шума: $\mu = 0, \sigma = 1$.

Параметры тренда (угол наклона): $\mu = 0, \sigma = 0.5$.

Для модели линейного тренда с изломами время между изломами имеет экспоненциальное распределение: $\tau \sim Exp(\lambda)$, где $\lambda$ --- среднее число изломов. В нашем случае $\lambda = \frac{1}{200}$, т.е. в среднем излом происходит один раз в 200 точек, а значит на один ряд в среднем приходится 5 изломов.

In [2]:
class SeriesGenerator():
    def __init__(self, noise_mean = 0, noise_sigma = 1, trend_mean = 0, trend_sigma = 0.5, length = 1000):
        self.noise_mean = noise_mean
        self.noise_sigma = noise_sigma
        self.trend_mean = trend_mean
        self.trend_sigma = trend_sigma
        self.length = length
    
    
    def white_noise(self):
        eps = np.random.normal(self.noise_mean, self.noise_sigma, self.length)
        ksi = np.cumsum(eps)
        return ksi
    
    def geometric_brownian_motion(self):
        eps = np.random.normal(self.noise_mean, self.noise_sigma, self.length)
        ksi = np.cumsum(eps)
        eps_ksi = np.exp(ksi)
        return eps_ksi
    
    def linear_trend(self):
        trend = np.random.normal(self.trend_mean, self.trend_sigma, 1) * np.arange(1, self.length + 1)
        return trend
    
    def trend_whiteNoise(self):
        trend = np.random.normal(self.trend_mean, self.trend_sigma, 1) * np.arange(1, self.length + 1)
        eps = np.random.normal(self.noise_mean, self.noise_sigma, self.length)
        ksi = np.cumsum(eps)
        return trend + ksi
    
    def difficultTrend_whiteNoise(self, exp_freq = 100):
        eps = np.random.normal(self.noise_mean, self.noise_sigma , self.length)
        ksi = np.cumsum(eps)
        frequency = np.random.exponential(exp_freq, 10)
        frequency = [int(round(x, 0)) for x in np.cumsum(frequency) if x < self.length]
        frequency.extend([self.length])
    
        trend = np.empty(shape = (self.length))
        range_start = 0
        
        for freq in frequency:
            current_coef = np.random.normal(self.trend_mean, self.trend_sigma, 1)[0]
            if range_start==0:
                trend[range_start:freq] = current_coef * np.arange(range_start + 1, freq + 1)
            else:
                trend[range_start:freq] = current_coef * np.arange(range_start + 1, freq + 1) + (trend[range_start - 1] - current_coef * (range_start - 1))
            
            range_start = freq
        
        generated_ts = ksi + trend
     
        return generated_ts    

In [10]:
white_noise = pd.read_csv('./series1/white_noise.csv')
geometric_brownian_motion = pd.read_csv('./series1/geometric_brownian_motion.csv')
linear_trend = pd.read_csv('./series1/linear_trend.csv')
trend_whiteNoise = pd.read_csv('./series1/trend_whiteNoise.csv')
difficultTrend_whiteNoise = pd.read_csv('./series1/difficultTrend_whiteNoise.csv')
difficult_trend = pd.read_csv('./series1/difficult_trend.csv')

# Preprocessing

In [3]:
class Preprocessing():
    def __init__(self, series, L = 991):
        self.series = series.values
        self.N = len(self.series)
        self.L = L
        self.K = self.N - self.L + 1
        self.test_size = 300
    
    def trajectory_matrix(self):
        for k in range(self.K):
            if k == 0:
                tr_m = self.series[k : k + self.L].reshape(self.L, 1)
            else:
                tr_m = np.hstack((tr_m, self.series[k : k + self.L].reshape(self.L,1)))
        return tr_m
    
    def preprocess(self):
        traj_matrix = self.trajectory_matrix()
        train_series = pd.DataFrame(traj_matrix[:-self.test_size, :])
        test_series = pd.DataFrame(traj_matrix[-self.test_size:, :])
        
        train_data = train_series.iloc[:,:-1]
        train_labels = train_series.iloc[:,-1]
        
        test_data = test_series.iloc[:,:-1]
        test_labels = test_series.iloc[:, -1]
        
        return train_data, test_data, train_labels, test_labels
        
    def set_L(self, L):
        self.L = L
        
    def differences(self):
        train_data, test_data, train_labels, test_labels = self.preprocess()
        train_data['labels'] = train_labels
        test_data['labels'] = test_labels
        
        train_data_diff = pd.DataFrame()
        test_data_diff = pd.DataFrame()
        
        for i in range(self.K - 1):
            train_data_diff[i] = train_data.iloc[:, i + 1] - train_data.iloc[:, i]
            test_data_diff[i] = test_data.iloc[:, i + 1] - test_data.iloc[:, i]
            
        train_data_diff, train_labels = train_data_diff.iloc[:, :-1], train_data_diff.iloc[:, -1]
        test_data_diff, test_labels_diff = test_data_diff.iloc[:, :-1], test_data_diff.iloc[:, -1]
        
        return train_data_diff, test_data_diff, train_labels, test_labels_diff, test_labels

# ParameterOptimization

In [4]:
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.neighbors import KNeighborsRegressor

In [6]:
#больше не нужен
#def lasso_cv(ts, alphas, k = 1, cv = 3, diff = False):
#    
#    prep = Preprocessing(ts, L = len(ts) - k)
#    if diff:
#        train_data, test_data, train_labels, test_labels_diff, test_labels = prep.differences()
#    else:
#        train_data, test_data, train_labels, test_labels = prep.preprocess()
#    
#    regr = LassoCV(alphas = alphas, cv = cv, n_jobs = -1, random_state = 42)
#    
#    regr.fit(train_data, train_labels)
#    
#    pred_train = regr.predict(train_data)
#    mse_train = mean_squared_error(pred_train, train_labels)
#    
#    pred_test = regr.predict(test_data)
#    if diff:
#        mse_test = mean_squared_error(pred_test, test_labels_diff)
#    else:
#        mse_test = mean_squared_error(pred_test, test_labels)
#    
#    return [mse_train, mse_test, pred_test, test_labels]

In [7]:
#больше не нужен
#def ridge_cv(ts, alphas, k = 1, cv = 3, diff = False):
#    prep = Preprocessing(ts, L = len(ts) - k)
#    if diff:
#        train_data, test_data, train_labels, test_labels_diff, test_labels = prep.differences()
#    else:
#        train_data, test_data, train_labels, test_labels = prep.preprocess()
#    
#    regr = RidgeCV(alphas = alphas, cv = cv, scoring = 'neg_mean_squared_error')
#    regr.fit(train_data, train_labels)
#    
#    pred_train = regr.predict(train_data)
#    mse_train = mean_squared_error(pred_train, train_labels)
#    
#    pred_test = regr.predict(test_data)
#    if diff:
#        mse_test = mean_squared_error(pred_test, test_labels_diff)
#    else:
#        mse_test = mean_squared_error(pred_test, test_labels)
#    
#    return [mse_train, mse_test, pred_test, test_labels]

In [8]:
#больше не нужен
#def linear_regression(ts, k=1, diff = False):
#    prep = Preprocessing(ts, L = len(ts) - k)
#    if diff:
#        train_data, test_data, train_labels, test_labels_diff, test_labels = prep.differences()
#    else:
#        train_data, test_data, train_labels, test_labels = prep.preprocess()
#        
#    regr = LinearRegression(fit_intercept=False)
#    regr.fit(train_data, train_labels)
#    
#    pred_train = regr.predict(train_data)
#    mse_train = mean_squared_error(pred_train, train_labels)
#    
#    pred_test = regr.predict(test_data)
#    if diff:
#        mse_test = mean_squared_error(pred_test, test_labels_diff)
#    else:
#        mse_test = mean_squared_error(pred_test, test_labels)
#    
#    return [mse_train, mse_test, pred_test, test_labels]

In [9]:
#больше не нужен
#def dt_cv(ts, dt_params, k=1, cv=3, diff = False):
#    prep = Preprocessing(ts, L = len(ts) - k)
#    if diff:
#        train_data, test_data, train_labels, test_labels_diff, test_labels = prep.differences()
#    else:
#        train_data, test_data, train_labels, test_labels = prep.preprocess()
#        
#    dt = DecisionTreeRegressor(random_state = 42)
#    grid = GridSearchCV(dt, dt_params, cv = cv, scoring = 'neg_mean_squared_error', n_jobs = -1)
#    
#    grid.fit(train_data, train_labels)
#    
#    pred_train = grid.predict(train_data)
#    mse_train = mean_squared_error(pred_train, train_labels)
#    
#    pred_test = grid.predict(test_data)
#    if diff:
#        mse_test = mean_squared_error(pred_test, test_labels_diff)
#    else:
#        mse_test = mean_squared_error(pred_test, test_labels)
#    
#    return [mse_train, mse_test, pred_test, test_labels]

In [10]:
#больше не нужен
#def knn_cv(ts, dt_params, k=1, cv=3, diff = False):
#    prep = Preprocessing(ts, L = len(ts) - k)
#    if diff:
#        train_data, test_data, train_labels, test_labels_diff, test_labels = prep.differences()
#    else:
#        train_data, test_data, train_labels, test_labels = prep.preprocess()
#        
#    knn = KNeighborsRegressor(random_state = 42)
#    grid = GridSearchCV(knn, knn_params, cv = cv, scoring = 'neg_mean_squared_error', n_jobs = -1)
#    
#    grid.fit(train_data, train_labels)
#    
#    pred_train = grid.predict(train_data)
#    mse_train = mean_squared_error(pred_train, train_labels)
#    
#    pred_test = grid.predict(test_data)
#    if diff:
#        mse_test = mean_squared_error(pred_test, test_labels_diff)
#    else:
#        mse_test = mean_squared_error(pred_test, test_labels)
#    
#    return [mse_train, mse_test, pred_test, test_labels]

In [42]:
def train_methods(ts, method, method_params = None, k = 1, cv = 3, intercept = False, diff = False):
    #параметр method может принимать значения 'linreg', 'lasso', 'ridge', 'dt', 'knn'
    prep = Preprocessing(ts, L = len(ts) - k)
    if diff:
        train_data, test_data, train_labels, test_labels_diff, test_labels = prep.differences()
    else:
        train_data, test_data, train_labels, test_labels = prep.preprocess()
        
    if method == 'linreg':
        selected_method = LinearRegression(fit_intercept = intercept)
    elif method == 'lasso':
        selected_method = LassoCV(alphas = method_params, cv = cv, n_jobs = -1, fit_intercept = intercept, random_state = 42)
    elif method == 'ridge':
        selected_method = RidgeCV(alphas = method_params, cv = cv, fit_intercept = intercept, scoring = 'neg_mean_squared_error')
    elif method == 'dt':
        dt = DecisionTreeRegressor(random_state = 42)
        selected_method = GridSearchCV(dt, method_params, cv = cv, scoring = 'neg_mean_squared_error', n_jobs = -1)
    elif method == 'knn':
        knn = KNeighborsRegressor(random_state = 42)
        selected_method = GridSearchCV(knn, method_params, cv = cv, scoring = 'neg_mean_squared_error', n_jobs = -1)
        
    selected_method.fit(train_data, train_labels)
    pred_train = selected_method.predict(train_data)
    mse_train = mean_squared_error(pred_train, train_labels)
    
    pred_test = selected_method.predict(test_data)
    if diff:
        mse_test = mean_squared_error(pred_test, test_labels_diff)
    else:
        mse_test = mean_squared_error(pred_test, test_labels)
        
    return mse_train, mse_test, pred_test, test_labels

In [43]:
class ParameterOptimization():
    def __init__(self, series):
        self.series = series
        
    def method_optimization(self, k_array, method, cv, method_params = None, diff = False, intercept = False):
        mse = []
        mse_tests = {}
        for k in k_array:
            mse_train, mse_test, pred_test, series_test = train_methods(self.series[:-300], method, method_params, k=k, 
                                                                        intercept = intercept, diff = diff)
            mse_tests[k] = mse_test
        best_k = list(mse_tests.keys())[list(mse_tests.values()).index(min(mse_tests.values()))]
        mse_train, mse_test, pred_test, series_test = train_methods(self.series, method, method_params, k = best_k, 
                                                                    intercept = intercept, diff = diff)
        return best_k, mse_train, mse_test
    
    #def linreg_optimization(self, k_array, diff = False, intercept = False):
    #    mse = []
    #    mse_tests = {}
    #    for k in k_array:
    #        mse_train, mse_test, pred_test, series_test = train_methods(self.series[:-300], 'linreg', k=k, 
    #                                                                    intercept = intercept, diff = diff)
    #        mse_tests[k] = mse_test
    #    best_k = list(mse_tests.keys())[list(mse_tests.values()).index(min(mse_tests.values()))]
    #    mse_train, mse_test, pred_test, series_test = train_methods(self.series, 'linreg', k = best_k, 
    #                                                                intercept = intercept, diff = diff)
    #    return best_k, mse_train, mse_test
    
    #def lasso_optimization(self, k_array, alphas, cv, diff = False, intercept = False):
    #    mse = []
    #    mse_tests = {}
    #    for k in k_array:
    #        mse_train, mse_test, pred_test, series_test = train_methods(self.series[:-300], 'lasso', alphas, 
    #                                                                    k = k, cv = cv, intercept = intercept, diff = diff)
    #        mse_tests[k] = mse_test
    #    best_k = list(mse_tests.keys())[list(mse_tests.values()).index(min(mse_tests.values()))]
    #    mse_train, mse_test, pred_test, series_test = train_methods(self.series, 'lasso', alphas, k = best_k, 
    #                                                                cv = cv, intercept = intercept, diff = diff)
    #    return best_k, mse_train, mse_test
    
    #def ridge_optimization(self, k_array, alphas, cv, diff = False, intercept = False):
    #    mse = []
    #    mse_tests = {}
    #    for k in k_array:
    #        mse_train, mse_test, pred_test, series_test = train_methods(self.series[:-300], 'ridge', alphas, 
    #                                                                    k = k, cv = cv, intercept = intercept, diff = diff)
    #        mse_tests[k] = mse_test
    #    best_k = list(mse_tests.keys())[list(mse_tests.values()).index(min(mse_tests.values()))]
    #    mse_train, mse_test, pred_test, series_test = train_methods(self.series, 'ridge', alphas, k = best_k, 
    #                                                                cv = cv, intercept = intercept, diff = diff)
    #    return best_k, mse_train, mse_test
    
    #def dt_optimization(self, k_array, dt_params, cv, diff = False, intercept = False):
    #    mse = []
    #    mse_tests = {}
    #    for k in k_array:
    #        mse_train, mse_test, pred_test, series_test = train_methods(self.series[:-300], 'dt', dt_params, 
    #                                                                    k = k, cv = cv, intercept = intercept, diff = diff)
    #        mse_tests[k] = mse_test
    #    best_k = list(mse_tests.keys())[list(mse_tests.values()).index(min(mse_tests.values()))]
    #    mse_train, mse_test, pred_test, series_test = train_methods(self.series, 'dt', dt_params, k = best_k, 
    #                                                                cv = cv, intercept = intercept, diff = diff)
    #    return best_k, mse_train, mse_test
    
    #def knn_optimization(self, k_array, knn_params, cv, diff = False):
    #    mse = []
    #    mse_tests = {}
    #    for k in k_array:
    #        mse_train, mse_test, pred_test, series_test = train_methods(self.series[:-300], 'knn', knn_params, 
    #                                                                    k = k, cv = cv, intercept = intercept, diff = diff)
    #        mse_tests[k] = mse_test
    #    best_k = list(mse_tests.keys())[list(mse_tests.values()).index(min(mse_tests.values()))]
    #    mse_train, mse_test, pred_test, series_test = train_methods(self.series, 'knn', knn_params, k = best_k, 
    #                                                                cv = cv, intercept = intercept, diff = diff)
    #    return best_k, mse_train, mse_test

## Линейный тренд

In [58]:
def get_scores(series_type, k_array, k_array_lr, method, method_params = None, cv = 3, diff = False, intercept = False):
    test_mse = []
    train_mse = []
    k_choosen = []
    for i in range(1, 101):
        opt = ParameterOptimization(series_type['series'+str(i)])
        if method == 'linreg':
            best_k, mse_train, mse_test = opt.method_optimization(k_array_lr, method, 5, method_params, diff, intercept)
        else:
            best_k, mse_train, mse_test = opt.method_optimization(k_array, method, 5, method_params, diff, intercept)
        
        test_mse.append(mse_test)
        train_mse.append(mse_train)
        k_choosen.append(best_k)
    
    return test_mse, train_mse, k_choosen

In [115]:
def get_results(series_type, methods, k_array, k_array_lr, method_params = None, cv = 3):
    #в качестве значения параметра methods нужно передать список из возможных моделей,
    #максимальный список --- ['linreg', 'lasso', 'ridge', 'dt', 'knn']
    result = pd.DataFrame()
    #methods = ['linreg', 'lasso', 'ridge', 'dt', 'knn']
    
    for method in methods:
        for diff in [True, False]:
            for intercept in [True, False]:
                if method == 'linreg':
                    test_mse, train_mse, k_choosen = get_scores(series_type, k_array, k_array_lr, method, method_params[method], 
                                                            cv, diff, intercept)
                else:
                    test_mse, train_mse, k_choosen = get_scores(series_type, k_array, k_array_lr, method, method_params[method], 
                                                            cv, diff, intercept)
                result = result.append({'Mean train MSE': np.mean(train_mse),
                                        'Mean test MSE': np.mean(test_mse),
                                        'Method': f'{method} (diff = {diff}, intercept = {intercept})',
                                        },
                                        ignore_index=True)
                
    return result

In [49]:
k_array = list(range(2, 100, 5))
alphas = np.append(0, 10**np.linspace(10,-2,100)*0.5)
dt_params = {'max_depth': range(1, 9)}
knn_params = {'n_neighbors': range(1, 10)}

method_params = {'lasso': alphas, 'ridge': alphas, 'linreg': None}

In [79]:
%time result_lt = get_results(linear_trend, ['lasso', 'ridge', 'linreg'], k_array, list(range(2, 100)), method_params, 5)
result_lt

,Mean test MSE,Mean train MSE,Method
0,6.015045e-28,1.133105e-28,"lasso (diff = True, intercept = True)"
1,1.933942e-27,3.955797e-28,"lasso (diff = True, intercept = False)"
2,1.256569e-27,3.723566e-28,"lasso (diff = False, intercept = True)"
3,3.640814e-02,8.859158e-03,"lasso (diff = False, intercept = False)"
4,5.992114e-28,1.134711e-28,"ridge (diff = True, intercept = True)"
5,6.625225e-28,1.203431e-28,"ridge (diff = True, intercept = False)"
6,4.830781e-27,9.683097e-28,"ridge (diff = False, intercept = True)"
7,1.188342e-26,2.790511e-27,"ridge (diff = False, intercept = False)"
8,6.000043e-28,1.137317e-28,"linreg (diff = True, intercept = True)"
9,7.575085e-28,1.358522e-28,"linreg (diff = True, intercept = False)"


## Шум

In [90]:
k_array = list(range(2, 100, 5))
alphas = np.append(0, 10**np.linspace(10,-2,100)*0.5)
dt_params = {'max_depth': range(1, 9)}
knn_params = {'n_neighbors': range(1, 10)}

method_params = {'lasso': alphas, 'ridge': alphas, 'linreg': None}

In [141]:
result_noise = get_results(white_noise, ['lasso', 'ridge', 'linreg'], k_array, list(range(2, 100)), method_params, 5)
result_noise

,Mean test MSE,Mean train MSE,Method
0,1.013945,0.983313,"lasso (diff = True, intercept = True)"
1,1.011624,0.985822,"lasso (diff = True, intercept = False)"
2,1.049827,0.975484,"lasso (diff = False, intercept = True)"
3,1.077941,1.016391,"lasso (diff = False, intercept = False)"
4,1.015794,0.972734,"ridge (diff = True, intercept = True)"
5,1.014919,0.970044,"ridge (diff = True, intercept = False)"
6,1.044066,0.974569,"ridge (diff = False, intercept = True)"
7,1.028636,0.978227,"ridge (diff = False, intercept = False)"
8,1.017127,0.993243,"linreg (diff = True, intercept = True)"
9,1.015029,0.991034,"linreg (diff = True, intercept = False)"


## Линейный тренд + шум

In [54]:
k_array = list(range(2, 100, 5))
alphas = np.append(0, 10**np.linspace(10,-2,100)*0.5)
dt_params = {'max_depth': range(1, 9)}
knn_params = {'n_neighbors': range(1, 10)}

method_params = {'lasso': alphas, 'ridge': alphas, 'linreg': None}

In [162]:
result_Trendnoise = get_results(trend_whiteNoise, ['lasso', 'ridge', 'linreg'], k_array, list(range(2, 100)), method_params, 5)
result_Trendnoise

,Mean test MSE,Mean train MSE,Method
0,1.012500,0.995646,"lasso (diff = True, intercept = True)"
1,1.048501,0.987633,"lasso (diff = True, intercept = False)"
2,2.431317,2.163391,"lasso (diff = False, intercept = True)"
3,2.198690,2.059733,"lasso (diff = False, intercept = False)"
4,1.014413,0.985847,"ridge (diff = True, intercept = True)"
5,1.032641,0.970047,"ridge (diff = True, intercept = False)"
6,1.037995,0.990298,"ridge (diff = False, intercept = True)"
7,1.066915,0.991251,"ridge (diff = False, intercept = False)"
8,1.015163,1.006181,"linreg (diff = True, intercept = True)"
9,1.062211,1.017119,"linreg (diff = True, intercept = False)"


## Сложный тренд с изломами

In [152]:
k_array = list(range(2, 100, 5))
alphas = np.append(0, 10**np.linspace(10,-2,100)*0.5)
dt_params = {'max_depth': range(1, 9)}
knn_params = {'n_neighbors': range(1, 10)}

method_params = {'lasso': alphas, 'ridge': alphas, 'linreg': None}

In [153]:
result_difficultTrend = get_results(difficult_trend, ['lasso', 'ridge', 'linreg'], k_array, list(range(2, 100)), method_params, 5)
result_difficultTrend

,Mean test MSE,Mean train MSE,Method
0,0.012319,0.002894,"lasso (diff = True, intercept = True)"
1,0.002703,0.003301,"lasso (diff = True, intercept = False)"
2,5.607846,1.248994,"lasso (diff = False, intercept = True)"
3,23.458689,3.218304,"lasso (diff = False, intercept = False)"
4,0.008180,0.002858,"ridge (diff = True, intercept = True)"
5,0.002604,0.002798,"ridge (diff = True, intercept = False)"
6,0.402696,0.003352,"ridge (diff = False, intercept = True)"
7,0.003385,0.002666,"ridge (diff = False, intercept = False)"
8,0.100542,0.002604,"linreg (diff = True, intercept = True)"
9,0.003495,0.002716,"linreg (diff = True, intercept = False)"


## Сложный тренд с изломами + шум

In [163]:
k_array = list(range(2, 100, 5))
alphas = np.append(0, 10**np.linspace(10,-2,100)*0.5)
dt_params = {'max_depth': range(1, 9)}
knn_params = {'n_neighbors': range(1, 10)}

method_params = {'lasso': alphas, 'ridge': alphas, 'linreg': None, 'dt': dt_params, 'knn': knn_params}

In [ ]:
result_difficultTrend_noise = get_results(difficultTrend_whiteNoise, ['lasso', 'ridge', 'linreg', 'dt', 'knn'], 
                                          k_array, list(range(2, 100)), method_params, 5)
result_difficultTrend_noise